In [1]:
# download covid tracking
# download state populations and FIPS
# iterate over states
    # compute fits
    # create dict with FIPS and doubling time
# feed growth rate to map

import requests
import pandas as pd
import numpy as np
import json

def download_covidtracking():
    URL = 'https://covidtracking.com/api/states/daily'
    r = requests.get(URL)
    sd = pd.read_json(json.dumps(r.json()))
    sd['date'] = pd.to_datetime(sd['date'], format='%Y%m%d')
    return sd

def download_FIPS():
    return pd.read_csv('https://raw.githubusercontent.com/dsoto/simple-covid-models/master/state-populations.csv',
                        index_col='USPS')

def fit_slopes(state_data):
    # list doubling times
    data_list = []
    for state, data in state_data.groupby('state'):
        # set date index
        data = data.set_index('date')
        # sort index
        data = data.sort_index()
        # discard older data
        days_data = 7
        data = data[-days_data:]
        fit_x = np.arange(0, days_data, 1)
        beta, log_intercept = np.polyfit(fit_x, np.log(data['positive']), 1)
        data_list.append({'state':state, 'beta':beta, 'doubling':np.log(2)/beta, 'daily_growth': np.exp(beta)-1})
    # return list of doubling times and USPS state codes in dictionary
    return pd.DataFrame(data_list)

def add_FIPS_to_fits(fit_data, FIPS_data):
    fit_data = fit_data.set_index('state')
    # hand remove states with no FIPS
    fit_data = fit_data.drop(['AS', 'MP', 'VI', 'GU', 'PR'])
    fit_data['FIPS'] = FIPS_data['FIPS']
    fit_data = fit_data.set_index('FIPS')
    # keys are FIPS, vals are data for color mapping
    return fit_data['daily_growth'].to_dict()

In [4]:
covid_data = download_covidtracking()
FIPS_data = download_FIPS()
fit_data = fit_slopes(covid_data)
color_data = add_FIPS_to_fits(fit_data, FIPS_data)

In [3]:
# plotting imports
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
import bqplot as bq
import ipywidgets as widgets
from IPython.display import display

sc_cl = bqs.ColorScale(scheme='viridis')
sc_geo = bqs.AlbersUSA(scale_factor=1200, translate=(500,400))
axis = bqa.ColorAxis(scale=sc_cl, num_ticks=5)
map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)

fig = bq.Figure(marks=[states_map], axes=[axis], title='COVID Daily Growth By State')
box = widgets.VBox([fig])
display(box)